In [15]:
import numpy as np

In [16]:
#Initialization of Weights and Creating a Random Dataset
np.random.seed(1)
A = np.random.randn(20, 32, 32,1)  #Dataset having 20 examples
Y=np.random.randint(9,size=20)   #True Labels
b = np.zeros((20,10))       #One-hot Encoding of True Labels
b[np.arange(20), Y] = 1
Y=b
W1 = np.random.randn(3, 3,1, 4)
W3 =np.random.randn(1024,10)

b1 = np.random.randn(1, 1,1, 4)
b3=np.random.randn(1,10)


In [17]:
Y

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.

In [18]:
def conv_forward(A_prev, W, b):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    stride = 2
    
    n_H = 16
    n_W = 16
    Z = np.zeros((m, n_H, n_W, n_C))
    for i in range(m):                                 # loop over the batch of training examples
        a_prev = A_prev[i]                     # Select ith training example's padded activation
        for h in range(n_H):                           # loop over vertical axis of the output volume
            for w in range(n_W):                       # loop over horizontal axis of the output volume
                for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = a_prev[vert_start:vert_end, horiz_start:horiz_end, :]
                    if a_slice_prev.shape==W[...,c].shape:
                        Z[i, h, w, c]=np.sum(np.multiply(a_slice_prev, W[...,c])+  b[...,c] )
                         #We have to slice the weights for the last stride both verically and horizontally to prevent any error to occur as some part of it will not overlap the input matrix
                    else:
                        if a_slice_prev.shape==(3,2,1):     
                            Z[i, h, w, c]=np.sum(np.multiply(a_slice_prev, W[:3,:2,:,c]) )#+  b[...,c])
                        if a_slice_prev.shape==(2,3,1):
                            Z[i, h, w, c]=np.sum(np.multiply(a_slice_prev, W[:2,:3,:,c]) +  b[...,c])
                        
                    
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b)
    
    return Z, cache

In [19]:
def conv_backward(dZ, cache):
    
    
  
    # Retrieve information from "cache"
    (A_prev, W, b) = cache
    
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    stride = 2
    (m, n_H, n_W, n_C) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    
    
    for i in range(m):                       # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev = A_prev[i]
        da_prev = dA_prev[i]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Update gradients for the window and the filter's parameters using the code formulas given above
                    da_prev[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        
        dA_prev[i, :, :, :] = da_prev
    
    return dA_prev, dW, db

In [20]:
def relu(Z):
   
    A = np.maximum(0,Z)  
    cache = Z 
    return A, cache

In [21]:
def relu_backward(dA, cache):
   
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, dZ should be also zero
    dZ[Z <= 0] = 0
    
    return dZ

In [22]:
def fc_forward(A, W, b):
    Z = np.dot(A,W) + b
    cache = (A, W, b)
    
    return Z, cache

In [23]:
def fc_backward(dZ,cache):
    A_prev, W, b = cache
    m = A_prev.shape[0]

    dW = np.dot( cache[0].T,dZ) / m
    #print(dW.shape)
    db = np.squeeze(np.sum(dZ, axis=0, keepdims=True)) / m
    #print(db.shape)
    dA_prev = np.dot( dZ,cache[1].T)
    
    return dA_prev, dW, db

In [24]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [29]:
def forward_model(A,Wc,Wf1,bc,bf1):
    caches = []
    Z,cache=conv_forward(A,Wc,bc)
    #print(Z.shape)
    Z=Z.reshape(20,-1)
    caches.append(cache)
    Z,cache=relu(Z)
    #print(Z.shape)
    caches.append(cache)
    Z,cache=fc_forward(Z,Wf1,bf1)
    #print(Z.shape)
    caches.append(cache)
    AL=softmax(Z)
    output=np.argmax(AL,axis=1)
    
    return AL,caches,output
    
    

In [30]:
def backprop_model(AL, Y, caches):
    grads = {}  #Dictionary to store gradients
    Y = Y.reshape(AL.shape)
    dAL = dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    grads["dA3"], grads["dW3"], grads["db3"] = fc_backward(dAL, caches[-1])
    dZ2=relu_backward(grads["dA3"],caches[-2])
    dZ2=dZ2.reshape((20,16,16,4))
    grads["dA1"],grads["dW1"],grads["db1"]=conv_backward(dZ2,caches[-3])
    
    return grads

In [31]:
def update_parameters(W1,b1,W3,b3, grads, lr=0.1):
    W1=W1-lr*grads["dW1"]
    b1=b1-lr*grads["db1"]
    W3=W3-lr*grads["dW3"]
    b3=b3-lr*grads["db3"]
    
    
        
    return W1,b1,W3,b3

In [33]:
for i in range(100):
    print(i)
    AL,caches,output=forward_model(A,W1,W3,b1,b3)
    grads=backprop_model(AL, Y, caches)
    W1,b1,W3,b3=update_parameters(W1,b1,W3,b3, grads, 0.1)
    

0


C:\Users\Vaibhav\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in maximum
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Vaibhav\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less_equal
  


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
